In [1]:
from numpy import mean
from numpy import std
from numpy import dstack
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.metrics import *
from tensorflow.keras.utils import to_categorical
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[2], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

4 Physical GPUs, 1 Logical GPU


In [3]:
s25 = np.load('s25.npy')
h25 = np.load('h25.npy')

In [4]:
s25_labels = np.array([0 for _ in range(0,len(s25))])
h25_labels = np.array([1 for _ in range(0,len(h25))])

In [5]:
X = np.append(s25,h25,axis=0)

In [6]:
y = np.append(s25_labels,h25_labels,axis=0)

In [7]:
X.shape

(1142, 6250, 19)

In [8]:
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 10, 32
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, input_shape=(n_timesteps,n_features)))
    model.add(LeakyReLU())
    model.add(Conv1D(filters=64, kernel_size=3))
    model.add(LeakyReLU())
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100))
    model.add(LeakyReLU())
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy',Precision(),Recall(),AUC()])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy,precision,recall,auc = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy,precision,recall,auc

In [9]:
def summarize_results(accuracies,precisions,recalls,aucs):
    m, s = mean(accuracies), std(accuracies)
    print( ' Accuracy: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(precisions), std(precisions)
    print( ' Precision: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(recalls), std(recalls)
    print( ' Recall: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(aucs), std(aucs)
    print( ' AUC: %.3f%% (+/-%.3f) ' % (m, s))

In [10]:
def run_experiment(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    # repeat experiment
    scores = list()
    accuracies = list()
    precisions = list()
    recalls = list()
    aucs = list()
    # repeat experiment
    scores = list()
    for r in range(repeats):
        accuracy,precision,recall,auc = evaluate_model(trainX, trainy, testX, testy)
        accuracy = accuracy * 100.0
        precision = precision * 100.0
        recall = recall * 100.0
        auc = auc * 100.0
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        aucs.append(auc)
    # summarize results
    summarize_results(accuracies,precisions,recalls,aucs)

In [11]:
run_experiment(X,y)

Epoch 1/10
29/29 [==============================] - 1s 20ms/step - loss: 145.1671 - accuracy: 0.5597 - precision: 0.5320 - recall: 0.5047 - auc: 0.5633
Epoch 2/10
29/29 [==============================] - 1s 20ms/step - loss: 5.3965 - accuracy: 0.8445 - precision: 0.8907 - recall: 0.7617 - auc: 0.8512
Epoch 3/10
29/29 [==============================] - 1s 19ms/step - loss: 2.3870 - accuracy: 0.9507 - precision: 0.9705 - recall: 0.9229 - auc: 0.9639
Epoch 4/10
29/29 [==============================] - 1s 20ms/step - loss: 0.4541 - accuracy: 0.9617 - precision: 0.9690 - recall: 0.9486 - auc: 0.9734
Epoch 5/10
29/29 [==============================] - 1s 20ms/step - loss: 1.4425 - accuracy: 0.9463 - precision: 0.9523 - recall: 0.9322 - auc: 0.9557
Epoch 6/10
29/29 [==============================] - 1s 20ms/step - loss: 0.3497 - accuracy: 0.9693 - precision: 0.9717 - recall: 0.9626 - auc: 0.9794
Epoch 7/10
29/29 [==============================] - 1s 20ms/step - loss: 0.0921 - accuracy: 0.9923

In [12]:
def evaluate_model_2(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 10, 32
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3,input_shape=(n_timesteps,n_features)))
    model.add(LeakyReLU())
    model.add(Dropout(0.5))
    model.add(Conv1D(filters=64, kernel_size=3))
    model.add(Dropout(0.5))
    model.add(Conv1D(filters=64, kernel_size=3))
    model.add(LeakyReLU())
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100))
    model.add(LeakyReLU())
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy',Precision(),Recall(),AUC()])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy,precision,recall,auc = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy,precision,recall,auc

In [13]:
def run_experiment_2(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    # repeat experiment
    scores = list()
    accuracies = list()
    precisions = list()
    recalls = list()
    aucs = list()
    # repeat experiment
    scores = list()
    for r in range(repeats):
        accuracy,precision,recall,auc = evaluate_model_2(trainX, trainy, testX, testy)
        accuracy = accuracy * 100.0
        precision = precision * 100.0
        recall = recall * 100.0
        auc = auc * 100.0
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        aucs.append(auc)
    # summarize results
    summarize_results(accuracies,precisions,recalls,aucs)

In [14]:
run_experiment_2(X,y)

Epoch 1/10
29/29 [==============================] - 1s 25ms/step - loss: 377.5270 - accuracy: 0.5126 - precision_5: 0.4745 - recall_5: 0.3692 - auc_5: 0.5046
Epoch 2/10
29/29 [==============================] - 1s 24ms/step - loss: 11.2171 - accuracy: 0.6123 - precision_5: 0.5877 - recall_5: 0.5794 - auc_5: 0.6203
Epoch 3/10
29/29 [==============================] - 1s 24ms/step - loss: 6.1279 - accuracy: 0.6736 - precision_5: 0.6617 - recall_5: 0.6215 - auc_5: 0.6931
Epoch 4/10
29/29 [==============================] - 1s 24ms/step - loss: 3.5147 - accuracy: 0.7689 - precision_5: 0.7653 - recall_5: 0.7313 - auc_5: 0.7932
Epoch 5/10
29/29 [==============================] - 1s 24ms/step - loss: 5.8300 - accuracy: 0.7755 - precision_5: 0.7866 - recall_5: 0.7150 - auc_5: 0.8045
Epoch 6/10
29/29 [==============================] - 1s 23ms/step - loss: 5.0588 - accuracy: 0.8028 - precision_5: 0.8054 - recall_5: 0.7640 - auc_5: 0.8214
Epoch 7/10
29/29 [==============================] - 1s 23ms/s

In [15]:
def evaluate_model_3(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 10, 32
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, input_shape=(n_timesteps,n_features)))
    model.add(LeakyReLU())
    model.add(Conv1D(filters=64, kernel_size=3))
    model.add(LeakyReLU())
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100))
    model.add(LeakyReLU())
    model.add(Dropout(0.25))
    model.add(Dense(50))
    model.add(LeakyReLU())
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy',Precision(),Recall(),AUC()])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy,precision,recall,auc = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy,precision,recall,auc

In [16]:
def run_experiment_3(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    # repeat experiment
    scores = list()
    accuracies = list()
    precisions = list()
    recalls = list()
    aucs = list()
    # repeat experiment
    scores = list()
    for r in range(repeats):
        accuracy,precision,recall,auc = evaluate_model_3(trainX, trainy, testX, testy)
        accuracy = accuracy * 100.0
        precision = precision * 100.0
        recall = recall * 100.0
        auc = auc * 100.0
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        aucs.append(auc)
    # summarize results
    summarize_results(accuracies,precisions,recalls,aucs)

In [17]:
run_experiment_3(X,y)

Epoch 1/10
29/29 [==============================] - 1s 19ms/step - loss: 137.6687 - accuracy: 0.5389 - precision_10: 0.5094 - recall_10: 0.4439 - auc_10: 0.5360
Epoch 2/10
29/29 [==============================] - 1s 19ms/step - loss: 5.5497 - accuracy: 0.7470 - precision_10: 0.7790 - recall_10: 0.6425 - auc_10: 0.7557
Epoch 3/10
29/29 [==============================] - 1s 19ms/step - loss: 3.9355 - accuracy: 0.8916 - precision_10: 0.8964 - recall_10: 0.8692 - auc_10: 0.9241
Epoch 4/10
29/29 [==============================] - 1s 18ms/step - loss: 0.9878 - accuracy: 0.9069 - precision_10: 0.9455 - recall_10: 0.8505 - auc_10: 0.9393
Epoch 5/10
29/29 [==============================] - 1s 19ms/step - loss: 0.5194 - accuracy: 0.9551 - precision_10: 0.9618 - recall_10: 0.9416 - auc_10: 0.9734
Epoch 6/10
29/29 [==============================] - 1s 18ms/step - loss: 0.3395 - accuracy: 0.9671 - precision_10: 0.9693 - recall_10: 0.9603 - auc_10: 0.9802
Epoch 7/10
29/29 [==========================